In [23]:
import pandas as pd
from sqlalchemy import create_engine

## Clean 2018 flight data

In [24]:
#This file is too big to upload to github.  Please download this file from https://www.kaggle.com/zernach/2018-airplane-flights
#this file is not included in the resources folder.
csv_file = "Cleaned_2018_Flights.csv"
airline_2018_df = pd.read_csv(csv_file)
airline_2018_df.head()

 

,Unnamed: 0,ItinID,MktID,MktCoupons,Quarter,Origin,OriginWac,Dest,DestWac,Miles,ContiguousUSA,NumTicketsOrdered,AirlineCompany,PricePerTicket
0,0,20181767585,2018176758501,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,672.87
1,1,20181767586,2018176758601,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,367.68
2,2,20181767587,2018176758701,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,417.94
3,3,20181767636,2018176763601,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,247.10
4,4,20181767637,2018176763701,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,276.35


In [25]:
airline_2018_df.columns

Index(['Unnamed: 0', 'ItinID', 'MktID', 'MktCoupons', 'Quarter', 'Origin',
       'OriginWac', 'Dest', 'DestWac', 'Miles', 'ContiguousUSA',
       'NumTicketsOrdered', 'AirlineCompany', 'PricePerTicket'],
      dtype='object')

In [26]:
cleaned_airline_2018_df = airline_2018_df[['Origin', 'Dest', 'Miles', 'AirlineCompany','PricePerTicket' ]]
cleaned_airline_2018_df.head()

,Origin,Dest,Miles,AirlineCompany,PricePerTicket
0,PHL,LAX,2402.0,AA,672.87
1,PHL,LAX,2402.0,AA,367.68
2,PHL,LAX,2402.0,AA,417.94
3,PHL,LAX,2402.0,AA,247.10
4,PHL,LAX,2402.0,AA,276.35


In [27]:
#rename columns

cleaned_airline_2018_df.columns =["origin_airport_code", "dest_airport_code", "miles", "airline_code",  "price" ]
cleaned_airline_2018_df.head()

,origin_airport_code,dest_airport_code,miles,airline_code,price
0,PHL,LAX,2402.0,AA,672.87
1,PHL,LAX,2402.0,AA,367.68
2,PHL,LAX,2402.0,AA,417.94
3,PHL,LAX,2402.0,AA,247.10
4,PHL,LAX,2402.0,AA,276.35


In [30]:
 # pip install psycopg2 
# rds_connection_string = "<insert user name>:<insert password>@localhost:5432/<insert database>"
rds_connection_string = "postgres:postgres@localhost:5432/airline_etl"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()


    
    

['airline_codes', 'airport_name', 'flight_2018']

In [ ]:
# Use pandas to load csv converted DataFrame into database
cleaned_airline_2018_df.to_sql(name='flight_2018', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from flight_2018', con=engine).head()

## Clean Airport Code Data

In [ ]:
csv_file = "resources/airport_code.csv"
airline_2018_df = pd.read_csv(csv_file)
airline_2018_df.head()
